In [ ]:
import afar
import pytest
from pytest import raises

In [ ]:
with afar.run(), locally:
    x = 1
    y = x + 1

In [ ]:
with afar.run(), afar.locally:
    pass

In [ ]:
with afar.run(), locally:
    pass

In [ ]:
with afar.run(), afar.locally:
    pass

In [ ]:
with raises(NameError, match="locallyblah"):
    with afar.run(), locallyblah:
        pass

In [ ]:
with raises(Exception, match="`afar.run` is missing a location"):
    with afar.run():
        pass

In [ ]:
z = 1

def f():
    w = 10
    with afar.run() as results, locally:
        x = z
        y = x + 1 + w
    return results

results = f()
assert "x" not in results
assert results["y"] == 12
assert not hasattr(results, "w")
assert not hasattr(results, "z")

In [ ]:
with afar.run as results, afar.locally:
    x = z
    y = x + 1

In [ ]:
with raises(NameError):
    x
assert results == {"y": 2}

In [ ]:
with \
    afar.run() as results, \
    locally \
:
    x = z
    y = x + 1
assert results == {'y': 2}

In [ ]:
data = {"a": 1}
run = afar.run(data=data)
with run, locally:
    b = a + 1

In [ ]:
assert run.data is data
assert data == {"a": 1, "b": 2}
c = 10

In [ ]:
with run, locally:
    d = a + b + c
assert data == {"a": 1, "b": 2, "d": 13}

In [ ]:
# singleton doesn't save data
with afar.run as data2, locally:
    e = 100
assert afar.run.data is None
assert data2 == {"e": 100}

In [ ]:
with afar.run as results, locally:
    a = 1
    b = (
        a
        +
        2
    )
assert results == {"b": 3}

In [ ]:
with afar.run as results, locally:
    a = 1
    b = (
        a
        +
        2
    )

In [ ]:
assert results == {"b": 3}

In [ ]:
results

In [ ]:
afar.run.context_body

In [ ]:
run = afar.run()
with run, later:
    1 / 0
assert run.context_body == ["    1 / 0\n"]

In [ ]:
with raises(RuntimeError, match="please put the context body on a new line"):
    with afar.run, later: pass

In [ ]:
with run, later:
    pass

assert run.context_body == ["    pass\n", "\n"]

In [ ]:
with raises(Exception, match="missing"):
    with run:
        pass

In [ ]:
with run, later:
    b = a + 1
    c = a + b

assert run.context_body == ["    b = a + 1\n", "    c = a + b\n", "\n"]

In [ ]:
with raises(Exception, match="missing"):
    with run:
        with later:
            pass

In [ ]:
# It would be nice if we could make these fail
with run, later as z:
    pass

with run, later as [z, *other]:
    pass

with run, later, z:
    pass

In [ ]:
with \
    run, \
    later \
:

    pass

# fmt: on

assert run.context_body == ["\n", "    pass\n", "\n", "# fmt: on\n", "\n"]

In [ ]:
with \
    run as c, \
    later( \
    d= \
    ":" \
    ) \
:

    f
    g
    h(
        z
        =
        2
    )

In [ ]:
assert run.context_body == [
    "\n",
    "    f\n",
    "    g\n",
    "    h(\n",
    "        z\n",
    "        =\n",
    "        2\n",
    "    )\n",
]

In [ ]:
with \
    run, \
    later:
    # :
    (
        1
        +
        2
    )
x = (
    3
    +
    4
)
assert run.context_body == [
    "    # :\n",
    "    (\n",
    "        1\n",
    "        +\n",
    "        2\n",
    "    )\n",
]

In [ ]:
from operator import add
from dask.distributed import Client

client = Client()
two = client.submit(add, 1, 1)

with afar.run as results, afar.remotely:
    three = two + 1
assert three.result() == 3

with afar.get, afar.remotely(priority=1):
    five = two + three
assert five == 5